In [ ]:
%pip install ipympl
#%matplotlib inline
#%matplotlib widget

Environment Setup+angle conversion

In [ ]:
# =========================================================
# Cell 1 — MonsterTruck environment setup (MuJoCo, radians)
# Utilities, constants, and policies ONLY (no collection here)
# =========================================================
import os, math, time
import numpy as np
import mujoco

# ------------------- Global config -------------------
SEED = 0
rng = np.random.default_rng(SEED)

XML_PATH   = "monstertruck.xml"   # update if needed
FRAME_SKIP = 10                   # sim steps per control
ACTION_LOW, ACTION_HIGH = -1.0, 1.0

# Sign convention: + when pitching toward the head direction
PITCH_SIGN = +1   # flip to -1 if your model's "forward" comes out negative

# Success hysteresis (now in radians)
SUCCESS_ENTER_RAD    = np.deg2rad(178.0)
SUCCESS_RELEASE_RAD  = np.deg2rad(176.5)
MAX_UPRIGHT_RATE_RAD = np.deg2rad(60.0)
HOLD_NEEDED          = 1

# ------------------- Angle helpers (radians) -------------------
def clip(v, lo, hi):
    return lo if v < lo else (hi if v > hi else v)

def angdiff_rad(curr, prev):
    """Minimal signed angle diff in radians, (-pi, pi]."""
    return ((curr - prev + np.pi) % (2.0 * np.pi)) - np.pi

def get_signed_flip_rad(data, body_id, roof_is_neg_z=True):
    """
    Signed flip angle φ [rad] with your convention:
      0 = upside-down, ±π = upright.
      Positive when pitching toward the head (forward), negative otherwise.

    Numerically-stable via atan2(||z×r||, z·r). Sign from (z×r)·y (about pitch axis).
    """
    R = data.xmat[body_id].reshape(3, 3)     # body->world rotation
    z_w = np.array([0.0, 0.0, 1.0], float)   # world up

    body_z_world = R[:, 2]
    r_w = -body_z_world if roof_is_neg_z else body_z_world  # roof normal in world
    y_w =  R[:, 1]  # body Y (pitch axis) expressed in world

    v = np.cross(z_w, r_w)                   # perpendicular to both
    c = float(np.dot(z_w, r_w))              # cos-like term
    s_axis = float(np.dot(v, y_w))           # sign wrt pitch axis (forward/backward)
    theta = math.atan2(np.linalg.norm(v), clip(c, -1.0, 1.0))  # [0, π] in radians

    # Persistent sign memory to avoid flicker near 0
    if not hasattr(get_signed_flip_rad, "_flip_sign"):
        get_signed_flip_rad._flip_sign = 0.0
    if theta > np.deg2rad(0.5) and abs(s_axis) > 1e-12:
        get_signed_flip_rad._flip_sign = 1.0 if (PITCH_SIGN * s_axis) > 0.0 else -1.0

    sgn = get_signed_flip_rad._flip_sign if get_signed_flip_rad._flip_sign != 0.0 else 1.0
    return sgn * theta

# ------------------- Reset / control helpers -------------------
def reset_state(model, data):
    """Upside-down spawn with tiny jitter."""
    mujoco.mj_resetData(model, data)
    data.qpos[:3] = np.array([0.0, 0.0, 0.25])  # slight above ground
    data.qvel[:]  = 0.0
    data.qpos[3:7] = np.array([0.0, 1.0, 0.0, 0.0])  # 180° about X (upside-down)
    mujoco.mj_forward(model, data)
    # reset flip-sign memory
    get_signed_flip_rad._flip_sign = 0.0

def find_throttle_ids(model):
    ids = []
    for name in ["front_left_motor", "front_right_motor",
                 "rear_left_motor", "rear_right_motor"]:
        aid = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, name)
        if aid != -1:
            ids.append(aid)
    if not ids:
        ids = list(range(model.nu))  # fallback: drive all actuators
    return ids

def apply_throttle(data, ids, u):
    if data.ctrl is not None and data.ctrl.size > 0:
        data.ctrl[:] = 0.0
        for aid in ids:
            data.ctrl[aid] = u

# ------------------- Continuous policies (use radians) -------------------
def random_policy():
    return float(rng.uniform(ACTION_LOW, ACTION_HIGH))

def swing_cont_policy(phi_rad, omega_rad, k=0.7, bias=0.0, noise_std=0.15):
    """
    Continuous 'swing' throttle (radians):
      - scales with angular rate via tanh(·)
      - optional small Gaussian exploration
    Returns: float in [-1, 1]
    """
    base = k * np.tanh(omega_rad / 3.0) + bias * np.sign(omega_rad)
    a = base + noise_std * rng.normal()
    return float(np.clip(a, ACTION_LOW, ACTION_HIGH))

# =========================================================
# Flip angle utilities for training (centralized here)
# =========================================================
class FlipTracker:
    """
    Per-env tracker that returns:
      - phi_rad          : upside-down–centered pitch in [-pi, pi]
      - omega_rad_s      : pitch rate (rad/s)
      - phi_unwrapped_rad: continuous accumulated angle (no wrapping)

    If a gyro sensor id is provided and a model is passed to step(),
    omega comes from the IMU gyro (imu_gyro) directly.
    Otherwise it falls back to finite-difference dphi/dt.
    """
    def __init__(self, dt_ctrl, roof_is_neg_z=True, gyro_id=None, pitch_axis_index=1):
        self.dt = float(dt_ctrl) if dt_ctrl > 0 else 1e-6
        self.roof_is_neg_z = bool(roof_is_neg_z)
        self.prev_phi = None
        self.unwrapped = 0.0
        self.gyro_id = gyro_id
        self.pitch_axis_index = int(pitch_axis_index)

    def reset(self):
        self.prev_phi = None
        self.unwrapped = 0.0
        # also reset the global sign-memory used by get_signed_flip_rad
        get_signed_flip_rad._flip_sign = 0.0

    def step(self, data, body_id, model=None):
        """
        Call after advancing physics for one control interval.
        If model and gyro_id are set, use IMU gyro for omega.
        """
        phi = get_signed_flip_rad(data, body_id, roof_is_neg_z=self.roof_is_neg_z)

        if self.prev_phi is None:
            dphi = 0.0
        else:
            dphi = angdiff_rad(phi, self.prev_phi)   # minimal signed diff ∈ (-pi, pi]

        self.unwrapped += dphi
        self.prev_phi = phi

        # Prefer IMU gyro if available & model is provided
        if (self.gyro_id is not None) and (model is not None):
            adr = model.sensor_adr[self.gyro_id]
            gyro = data.sensordata[adr : adr + 3]   # [wx, wy, wz]
            omega = float(gyro[self.pitch_axis_index]) * PITCH_SIGN
        else:
            # Fallback: finite-difference
            omega = dphi / self.dt

        return float(phi), float(omega), float(self.unwrapped)

def ud_pitch_deg_from_phi(phi_rad):
    """Convert φ to degrees in [-180, 180], where 0°=upside-down, ±180°=upright."""
    deg = ((math.degrees(float(phi_rad)) + 180.0) % 360.0) - 180.0
    return float(deg)

def get_ud_pitch_deg(data, body_id, roof_is_neg_z=True):
    """Direct degrees readout from MuJoCo state using your φ definition."""
    return ud_pitch_deg_from_phi(get_signed_flip_rad(data, body_id, roof_is_neg_z=roof_is_neg_z))

def make_flip_tracker(model, frame_skip=FRAME_SKIP, gyro_sensor_name="imu_gyro"):
    """
    Factory using env control interval dt_ctrl = model.opt.timestep * FRAME_SKIP.
    Tries to bind to imu_gyro so FlipTracker can use true angular velocity.
    """
    dt_ctrl = model.opt.timestep * frame_skip
    gyro_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SENSOR, gyro_sensor_name)
    if gyro_id < 0:
        gyro_id = None  # fallback: no gyro
    return FlipTracker(dt_ctrl=dt_ctrl, roof_is_neg_z=True, gyro_id=gyro_id, pitch_axis_index=1)


Tester for environment setup

In [ ]:
# =========================================================
# Cell 2 — Debug viewer: UD pitch (degrees) + pitch rate
# Keys: W=+throttle, S=−throttle, SPACE=0, ESC=quit
# =========================================================
import os, time, math
import numpy as np
import mujoco
from mujoco.glfw import glfw

# --- Load model, data, and set initial state using Cell 1 helpers ---
if not os.path.exists(XML_PATH):
    raise FileNotFoundError(f"Cannot find XML_PATH: {XML_PATH}")
model = mujoco.MjModel.from_xml_path(XML_PATH)
data  = mujoco.MjData(model)
reset_state(model, data)

# Body / actuators
chassis_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "chassis")
if chassis_id == -1:
    chassis_id = 0
throttle_ids = find_throttle_ids(model)
print(f"Driving actuators: {[mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR, i) for i in throttle_ids]}")
print(f"Initial chassis COM z: {data.xpos[chassis_id,2]:.3f} m")

# --- IMU gyro sensor (for pitch rate) ---
gyro_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SENSOR, "imu_gyro")
if gyro_id < 0:
    raise RuntimeError("imu_gyro sensor not found in model XML")
gyro_adr = model.sensor_adr[gyro_id]  # start index in data.sensordata

# --- Viewer setup ---
if not glfw.init():
    raise RuntimeError("GLFW init failed")
window = glfw.create_window(1000, 800, "MonsterTruck — UD Pitch (degrees)", None, None)
if not window:
    glfw.terminate()
    raise RuntimeError("GLFW window creation failed")
glfw.make_context_current(window)
glfw.swap_interval(1)  # vsync

cam = mujoco.MjvCamera()
opt = mujoco.MjvOption()
scene = mujoco.MjvScene(model, maxgeom=10000)
context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)
cam.distance, cam.elevation, cam.azimuth = 3.0, -25, 90

# --- Keyboard throttle (manual) ---
throttle = 0.0
def key_callback(window, key, scancode, action, mods):
    global throttle
    if action in (glfw.PRESS, glfw.REPEAT):
        if key == glfw.KEY_W:       throttle = +1.0
        elif key == glfw.KEY_S:     throttle = -1.0
        elif key == glfw.KEY_SPACE: throttle = 0.0
    elif action == glfw.RELEASE and key in (glfw.KEY_W, glfw.KEY_S):
        throttle = 0.0
glfw.set_key_callback(window, key_callback)

# --- Timing ---
dt_ctrl = model.opt.timestep * FRAME_SKIP  # one control interval (for reference)
last_print = time.perf_counter()
sim_start_wall = last_print
REALTIME = True

print("Controls: W=+throttle | S=−throttle | SPACE=0 | ESC=quit")
print("Readout: UD-pitch (deg) 0°=upside-down, ±180°=upright; ω = pitch rate (deg/s).")

while not glfw.window_should_close(window):
    if glfw.get_key(window, glfw.KEY_ESCAPE) == glfw.PRESS:
        break

    # Apply same throttle to all selected actuators
    apply_throttle(data, throttle_ids, throttle)

    # Step physics faster than render
    for _ in range(FRAME_SKIP):
        mujoco.mj_step(model, data)

    # Real-time pacing
    if REALTIME:
        target_wall = sim_start_wall + data.time
        now = time.perf_counter()
        if now < target_wall:
            time.sleep(target_wall - now)

    # Follow camera
    cam.lookat[:] = data.xpos[chassis_id]

    # Render
    w, h = glfw.get_framebuffer_size(window)
    if w > 0 and h > 0:
        mujoco.mjv_updateScene(model, data, opt, None, cam, mujoco.mjtCatBit.mjCAT_ALL, scene)
        viewport = mujoco.MjrRect(0, 0, w, h)
        mujoco.mjr_render(viewport, scene, context)
    glfw.swap_buffers(window)
    glfw.poll_events()

    # --- Read UD pitch (from custom φ) ---
    phi_rad = get_signed_flip_rad(data, chassis_id, roof_is_neg_z=True)
    ud_pitch_deg = ud_pitch_deg_from_phi(phi_rad)  # [-180, 180], 0=UD, ±180=UPRIGHT

    # --- Pitch rate from IMU gyro at imu_site ---
    # imu_gyro is angular velocity in IMU local frame: [wx, wy, wz] (rad/s)
    gyro = data.sensordata[gyro_adr : gyro_adr + 3]
    # Assume local Y is the pitch axis; flip sign with PITCH_SIGN if needed
    omega_rad = float(gyro[1]) * PITCH_SIGN
    omega_deg = math.degrees(omega_rad)

    # Console readout (~4 Hz)
    now = time.perf_counter()
    if now - last_print > 0.25:
        speed_ratio = data.time / (now - sim_start_wall + 1e-12)
        pos = data.xpos[chassis_id]
        print(
            f"t={data.time:6.2f}s  pos=({pos[0]:+.2f},{pos[1]:+.2f},{pos[2]:+.2f})  "
            f"UD-pitch={ud_pitch_deg:+7.2f}°  ω={omega_deg:+7.1f}°/s  "
            f"thr={throttle:+.2f}  ×{speed_ratio:.2f}"
        )
        last_print = now

glfw.terminate()
print("Debug viewer terminated.")


Data Collection for GP

In [ ]:
# =========================================================
# Cell 3 — Occupancy-aware random data collection for GP
# - GP model: X = (phi, omega_clipped) -> Y = (dphi, domega)
# - Random throttle + occasional resets (like MountainCar)
# - Each random action is held for MIN_HOLD_CTRL_STEPS
# - State-space binning in (phi, omega) with per-bin cap.
#   TOTAL_CTRL_STEPS is fixed; excess samples in dense bins
#   are discarded automatically.
# - At the end: print stats + plot bin occupancy and samples.
# =========================================================

import os, time, math
import numpy as np
import mujoco
import matplotlib.pyplot as plt

# ---------- Config ----------
TOTAL_CTRL_STEPS = 20000      # total control decisions (steps) to run
EPISODE_LEN_MAX  = 2000        # max control steps per episode before reset

# Clip angular velocity for GP inputs
OMEGA_CLIP_DEG   = 200.0
OMEGA_CLIP_RAD   = np.deg2rad(OMEGA_CLIP_DEG)

# Small prime after reset so it actually moves
PRIME_CTRL_STEPS = 30
PRIME_AMPL       = 0.7

# Minimum time to hold each random input before changing
MIN_HOLD_CTRL_STEPS = 1

PRINT_EVERY      = 1000        # progress print interval (in *saved* samples)


# ---------- Occupancy grid over (phi, omega) ----------
# We bin phi ∈ [-π, π], omega ∈ [-OMEGA_CLIP_RAD, +OMEGA_CLIP_RAD]
# and only keep up to MAX_SAMPLES_PER_BIN per bin.
N_BINS_PHI          = 30
N_BINS_OMEGA        = 30
MAX_SAMPLES_PER_BIN = 30

PHI_MIN, PHI_MAX = -math.pi, math.pi
OMEGA_MIN, OMEGA_MAX = -OMEGA_CLIP_RAD, OMEGA_CLIP_RAD

# occupancy[i_phi, i_omega] = number of accepted samples in that bin
occupancy = np.zeros((N_BINS_PHI, N_BINS_OMEGA), dtype=np.int32)


def state_to_bin(phi: float, omega_clipped: float):
    """
    Map (phi, omega_clipped) to integer bin indices (i_phi, i_omega).
    Both indices are in [0, N_BINS_* - 1].
    """
    # Clip to bounding box
    phi_c = float(np.clip(phi, PHI_MIN, PHI_MAX))
    om_c  = float(np.clip(omega_clipped, OMEGA_MIN, OMEGA_MAX))

    # Normalize to [0, 1]
    phi_norm = (phi_c - PHI_MIN) / (PHI_MAX - PHI_MIN + 1e-8)
    om_norm  = (om_c  - OMEGA_MIN) / (OMEGA_MAX - OMEGA_MIN + 1e-8)

    # Convert to bin indices
    i_phi = int(phi_norm * N_BINS_PHI)
    i_om  = int(om_norm  * N_BINS_OMEGA)

    # Clamp to valid indices
    i_phi = max(0, min(N_BINS_PHI   - 1, i_phi))
    i_om  = max(0, min(N_BINS_OMEGA - 1, i_om))
    return i_phi, i_om


# ---------- Load model & data ----------
if not os.path.exists(XML_PATH):
    raise FileNotFoundError(f"[Cell 3] Cannot find XML_PATH: {XML_PATH}")
model = mujoco.MjModel.from_xml_path(XML_PATH)
data  = mujoco.MjData(model)

# Control timing + flip tracker (uses imu_gyro if present)
dt_ctrl      = model.opt.timestep * FRAME_SKIP
flip_tracker = make_flip_tracker(model, frame_skip=FRAME_SKIP)

# Body / actuators
chassis_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "chassis")
if chassis_id == -1:
    chassis_id = 0
throttle_ids = find_throttle_ids(model)

print(f"[Cell 3] Driving actuators: "
      f"{[mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR, i) for i in throttle_ids]}")
print(f"[Cell 3] FRAME_SKIP={FRAME_SKIP}, dt_ctrl={dt_ctrl:.4f}s")
print(f"[Cell 3] Running for TOTAL_CTRL_STEPS={TOTAL_CTRL_STEPS} control steps.")
print(f"[Cell 3] Occupancy grid: {N_BINS_PHI} × {N_BINS_OMEGA}, "
      f"max {MAX_SAMPLES_PER_BIN} samples per bin.")


# ---------- Episode reset (MC-style) ----------
def reset_episode():
    """
    Reset to upside-down, prime a bit so it moves, then return initial (phi, omega).
    Omega comes from flip_tracker, which uses imu_gyro if available.
    """
    reset_state(model, data)
    flip_tracker.reset()

    # brief priming with random sign throttle so it actually swings
    for _ in range(PRIME_CTRL_STEPS):
        sgn = 1.0 if rng.uniform() < 0.5 else -1.0
        apply_throttle(data, throttle_ids, PRIME_AMPL * sgn)
        for _ in range(FRAME_SKIP):
            mujoco.mj_step(model, data)

    # sync tracker after priming
    flip_tracker.reset()
    phi0, omega0, _ = flip_tracker.step(data, chassis_id, model=model)
    return float(phi0), float(omega0)


# ---------- Storage for GP training ----------
phi_list      = []   # current φ (rad)
omega_list    = []   # current ω (rad/s, clipped)
u_list        = []   # throttle u (logged but not used as GP input)
dphi_list     = []   # Δφ over one control step (rad)
domega_list   = []   # Δω over one control step (rad/s)

# Track throttle range
u_min = +np.inf
u_max = -np.inf

# Success hysteresis (reuse from Cell 1 globals if present)
SUCCESS_ENTER_RAD    = globals().get("SUCCESS_ENTER_RAD",    np.deg2rad(178.0))
SUCCESS_RELEASE_RAD  = globals().get("SUCCESS_RELEASE_RAD",  np.deg2rad(176.5))
MAX_UPRIGHT_RATE_RAD = globals().get("MAX_UPRIGHT_RATE_RAD", np.deg2rad(60.0))
HOLD_NEEDED          = globals().get("HOLD_NEEDED", 1)


# ---------- Main collection loop (random policy + occupancy filter) ----------
phi_prev, omega_prev = reset_episode()

hold_success      = 0
saved_samples     = 0   # number of *accepted* transitions
episode_step      = 0
ctrl_steps_done   = 0
start_wall        = time.perf_counter()

# State for "minimum input time"
u_curr            = 0.0
hold_steps_left_u = 0    # countdown until we are allowed to change u_curr

print("[Cell 3] Starting RANDOM data collection (MC-style, occupancy-aware)...")

while ctrl_steps_done < TOTAL_CTRL_STEPS:
    # Episode reset if too long
    if episode_step >= EPISODE_LEN_MAX:
        phi_prev, omega_prev = reset_episode()
        episode_step   = 0
        hold_success   = 0
        u_curr         = 0.0
        hold_steps_left_u = 0

    # ----- Random action with minimum hold time -----
    if hold_steps_left_u <= 0:
        u_curr = float(rng.uniform(ACTION_LOW, ACTION_HIGH))
        hold_steps_left_u = MIN_HOLD_CTRL_STEPS
    hold_steps_left_u -= 1

    # Track throttle range (actual command space explored)
    u_min = min(u_min, u_curr)
    u_max = max(u_max, u_curr)

    # ----- Apply action and step physics for one control interval -----
    apply_throttle(data, throttle_ids, u_curr)
    for _ in range(FRAME_SKIP):
        mujoco.mj_step(model, data)

    ctrl_steps_done += 1

    # Read new state via flip tracker (uses imu_gyro for omega if configured)
    phi_curr, omega_curr, _ = flip_tracker.step(data, chassis_id, model=model)

    # Compute Δφ and Δω (raw)
    dphi   = angdiff_rad(phi_curr, phi_prev)
    domega = omega_curr - omega_prev

    # Clip ω input for GP stability (use previous-state omega)
    omega_clipped = float(np.clip(omega_prev, -OMEGA_CLIP_RAD, OMEGA_CLIP_RAD))

    # ----- Occupancy filter: decide whether to save this transition -----
    # We index bins by the *previous* state (φ_t, ω_t) which is what goes into X.
    i_phi, i_om = state_to_bin(phi_prev, omega_clipped)
    bin_count = occupancy[i_phi, i_om]

    if bin_count < MAX_SAMPLES_PER_BIN:
        # Accept sample and update occupancy
        occupancy[i_phi, i_om] += 1

        phi_list.append(float(phi_prev))
        omega_list.append(float(omega_clipped))  # clipped input to GP
        u_list.append(float(u_curr))             # logged, not used in X
        dphi_list.append(float(dphi))
        domega_list.append(float(domega))

        saved_samples += 1

        # Periodic progress print based on *saved* samples
        if (saved_samples % PRINT_EVERY == 0):
            elapsed = time.perf_counter() - start_wall
            phi_deg_arr   = np.rad2deg(np.asarray(phi_list,   dtype=np.float32))
            omega_deg_arr = np.rad2deg(np.asarray(omega_list, dtype=np.float32))

            unique_bins = np.count_nonzero(occupancy)
            print(
                f"[Cell 3] saved={saved_samples}  (ctrl_steps={ctrl_steps_done}/{TOTAL_CTRL_STEPS})  |  "
                f"φ range (deg) ≈ [{phi_deg_arr.min():+.1f}, {phi_deg_arr.max():+.1f}]  |  "
                f"ω range (deg/s, clipped) ≈ [{omega_deg_arr.min():+.1f}, {omega_deg_arr.max():+.1f}]  |  "
                f"u_range=[{u_min:+.2f}, {u_max:+.2f}]  |  "
                f"unique_bins={unique_bins}  |  elapsed={elapsed:.1f}s"
            )

    # ---------- Success hysteresis → reset (for diversity) ----------
    if (abs(phi_curr) >= SUCCESS_ENTER_RAD) and (abs(omega_curr) <= MAX_UPRIGHT_RATE_RAD):
        hold_success += 1
    elif abs(phi_curr) < SUCCESS_RELEASE_RAD:
        hold_success = 0

    if hold_success >= HOLD_NEEDED:
        phi_prev, omega_prev = reset_episode()
        episode_step   = 0
        hold_success   = 0
        # reset hold state too
        u_curr             = 0.0
        hold_steps_left_u  = 0
        continue

    # Prepare for next step
    phi_prev, omega_prev = phi_curr, omega_curr
    episode_step        += 1

print("[Cell 3] Finished random collection.")
print(f"[Cell 3] Total control steps taken: {ctrl_steps_done}")
print(f"[Cell 3] Total saved samples: {saved_samples}")


# ---------- Pack into arrays (Δstate model with X = [φ, ω]) ----------
phi_arr    = np.asarray(phi_list,    dtype=np.float32)
omega_arr  = np.asarray(omega_list,  dtype=np.float32)  # already clipped
u_arr      = np.asarray(u_list,      dtype=np.float32)
dphi_arr   = np.asarray(dphi_list,   dtype=np.float32)
domega_arr = np.asarray(domega_list, dtype=np.float32)

# GP input: X = [phi, omega_clipped]
X = np.stack([phi_arr, omega_arr], axis=1)         # (Nsaved, 2)
# GP target: Y = [Δphi, Δomega]
Y = np.stack([dphi_arr, domega_arr], axis=1)       # (Nsaved, 2)

print(f"[Cell 3] Final X shape = {X.shape}, Y shape = {Y.shape}")
print(f"[Cell 3] Final φ range (deg) ≈ [{np.rad2deg(phi_arr).min():+.1f}, {np.rad2deg(phi_arr).max():+.1f}]")
print(f"[Cell 3] Final ω range (deg/s, clipped) ≈ [{np.rad2deg(omega_arr).min():+.1f}, {np.rad2deg(omega_arr).max():+.1f}]")
print(f"[Cell 3] Final throttle range: u ∈ [{u_min:+.3f}, {u_max:+.3f}]")
print(f"[Cell 3] Unique occupied bins: {np.count_nonzero(occupancy)} "
      f"out of {N_BINS_PHI * N_BINS_OMEGA}.")


# ---------- Visualization: bin occupancy + sample distribution ----------
phi_deg   = np.rad2deg(phi_arr)
omega_deg = np.rad2deg(omega_arr)

PHI_MIN_DEG, PHI_MAX_DEG = -180.0, 180.0
OMEGA_MIN_DEG, OMEGA_MAX_DEG = -OMEGA_CLIP_DEG, OMEGA_CLIP_DEG

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 1) Heatmap of occupancy
ax0 = axes[0]
im = ax0.imshow(
    occupancy.T,
    origin="lower",
    aspect="auto",
    extent=[PHI_MIN_DEG, PHI_MAX_DEG, OMEGA_MIN_DEG, OMEGA_MAX_DEG],
)
ax0.set_xlabel("phi (deg)")
ax0.set_ylabel("omega (deg/s)")
ax0.set_title("Bin occupancy (count per bin)")
fig.colorbar(im, ax=ax0, label="count")

# 2) Scatter of saved samples
ax1 = axes[1]
ax1.scatter(phi_deg, omega_deg, s=5, alpha=0.4)
ax1.set_xlabel("phi (deg)")
ax1.set_ylabel("omega (deg/s)")
ax1.set_title("Saved samples in (phi, omega) space")
ax1.set_xlim(PHI_MIN_DEG, PHI_MAX_DEG)
ax1.set_ylim(OMEGA_MIN_DEG, OMEGA_MAX_DEG)

fig.tight_layout()
plt.show()


# ---------- Save to disk ----------
np.savez_compressed(
    "monstertruck_gp_initial_data.npz",
    X=X, Y=Y,
    phi=phi_arr, omega=omega_arr, u=u_arr,
    dphi=dphi_arr, domega=domega_arr,
    dt_ctrl=dt_ctrl,
    u_min=u_min, u_max=u_max,
    omega_clip_deg=OMEGA_CLIP_DEG,
)
print("[Cell 3] Saved dataset to 'monstertruck_gp_initial_data.npz'")


Visulisation of collected data

In [ ]:
# =========================================================
# Cell 4 — Phase-plane visualization of collected data
#   - Scatter: φ vs ω colored by throttle u
#   - Histograms: φ and ω coverage
# =========================================================
import numpy as np
import matplotlib.pyplot as plt

# ---------- Load data (from memory or from file) ----------
if 'phi_arr' in globals() and 'omega_arr' in globals() and 'u_arr' in globals():
    phi = phi_arr
    omega = omega_arr
    u = u_arr
    print("[Cell 4] Using phi_arr/omega_arr/u_arr from memory.")
else:
    data = np.load("monstertruck_gp_initial_data.npz")
    phi = data["phi"]      # radians
    omega = data["omega"]  # radians/s (clipped)
    u = data["u"]
    print("[Cell 4] Loaded data from 'monstertruck_gp_initial_data.npz'.")

N = phi.shape[0]
phi_deg   = np.rad2deg(phi)
omega_deg = np.rad2deg(omega)

print(f"[Cell 4] N samples = {N}")
print(f"[Cell 4] φ range (deg) ≈ [{phi_deg.min():+.1f}, {phi_deg.max():+.1f}]")
print(f"[Cell 4] ω range (deg/s) ≈ [{omega_deg.min():+.1f}, {omega_deg.max():+.1f}]")
print(f"[Cell 4] u range        ≈ [{u.min():+.3f}, {u.max():+.3f}]")


# ---------- Scatter: phase plane with action color ----------
MAX_POINTS = 50_000
rng_vis = np.random.default_rng(0)
if N > MAX_POINTS:
    idx = rng_vis.choice(N, size=MAX_POINTS, replace=False)
    N_show = MAX_POINTS
else:
    idx = slice(None)
    N_show = N

fig, ax = plt.subplots(figsize=(7.5, 6))
sc = ax.scatter(
    phi_deg[idx],
    omega_deg[idx],
    c=u[idx],
    s=8,
    cmap="coolwarm",
    alpha=0.7,
    edgecolors="none",
)

ax.set_xlabel("flip angle ϕ (deg)  —  0 = upside-down, ±180 = upright")
ax.set_ylabel("pitch rate ω (deg/s)")
ax.set_title(f"MonsterTruck data — phase plane (showing {N_show:,} / {N:,} samples)")

# vertical line at upside-down and (optionally) upright bands
ax.axvline(0.0, color="k", linestyle="--", alpha=0.25, linewidth=1.0)
ax.grid(True, alpha=0.3, linestyle="--", linewidth=0.8)

cb = fig.colorbar(sc, ax=ax, pad=0.02)
cb.set_label("throttle u")

plt.tight_layout()
plt.show()


# ---------- Histograms: φ and ω marginal distributions ----------
fig, axes = plt.subplots(1, 2, figsize=(10, 3.5))

axes[0].hist(phi_deg, bins=60, alpha=0.8)
axes[0].set_xlabel("flip angle ϕ (deg)")
axes[0].set_ylabel("count")
axes[0].set_title("Histogram of ϕ")

axes[1].hist(omega_deg, bins=60, alpha=0.8)
axes[1].set_xlabel("pitch rate ω (deg/s)")
axes[1].set_ylabel("count")
axes[1].set_title("Histogram of ω")

for ax in axes:
    ax.grid(True, alpha=0.25, linestyle="--", linewidth=0.7)

plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# Cell 5 — GP dynamics model for MonsterTruck (bin-thinned)
#   - Models: (phi, omega_clipped) -> (Δphi, Δomega)
#   - Two separate ExactGPs (one per output dim)
#   - Normalization of X, Y
#   - Numpy wrapper: gp_predict_delta_numpy(X_query)
#   - Supports incremental retraining via append_and_fit(...)
# ============================================================
import math
import numpy as np
import torch
import gpytorch

# ---------- Load data (X, Y) ----------
# Prefer in-memory X, Y from Cell 3; fall back to npz
if 'X' in globals() and 'Y' in globals():
    X_np = X.astype(np.float32)
    Y_np = Y.astype(np.float32)
    print("[Cell 5] Using X, Y from memory.")
else:
    data = np.load("monstertruck_gp_initial_data.npz")
    X_np = data["X"].astype(np.float32)  # (N, 2): [phi, omega_clipped]
    Y_np = data["Y"].astype(np.float32)  # (N, 2): [dphi, domega]
    print("[Cell 5] Loaded 'monstertruck_gp_initial_data.npz'.")

N, Dx = X_np.shape
_, Dy = Y_np.shape
assert Dx == 2 and Dy == 2, f"Unexpected shapes: X={X_np.shape}, Y={Y_np.shape}"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[Cell 5] Using device: {device}")


# ---------- Simple standardizer ----------
class StandardScaler:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, x: np.ndarray):
        self.mean = x.mean(axis=0, keepdims=True).astype(np.float32)
        self.std = x.std(axis=0, keepdims=True).astype(np.float32)
        # Avoid division by zero
        self.std[self.std < 1e-8] = 1.0

    def transform(self, x: np.ndarray) -> np.ndarray:
        return (x - self.mean) / self.std

    def inverse_transform(self, x: np.ndarray) -> np.ndarray:
        return x * self.std + self.mean


# ---------- Exact GP model (single-output) ----------
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel='RBF', ard_dims=None):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        if kernel == 'RBF':
            base_kernel = gpytorch.kernels.RBFKernel(ard_num_dims=ard_dims)
        else:
            base_kernel = gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=ard_dims)
        self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# ---------- GP manager for one output dimension ----------
class GPManager:
    def __init__(self, kernel='RBF', lr=0.05, iters=250, seed: int = 0):
        self.kernel = kernel
        self.lr = lr
        self.iters = iters

        self.x_scaler = StandardScaler()
        self.y_scaler = StandardScaler()

        self.model = None
        self.likelihood = None

        # Full dataset (grow over time, used for logging/visualisation)
        self.X_all = None
        self.Y_all = None

        # RNG (kept for future use if we ever add subsampling)
        self._rng = np.random.default_rng(seed)

    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        Train GP on given X, y (numpy arrays).
        Uses ALL provided points. (State-space binning was already
        done in Cell 3, so no further subsampling here.)
        """
        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float32).reshape(-1, 1)

        # Store full dataset for later analysis/visualization
        self.X_all = X.copy()
        self.Y_all = y.copy()

        # Fit scalers on full dataset
        self.x_scaler.fit(X)
        self.y_scaler.fit(y)

        Xn = self.x_scaler.transform(X)
        yn = self.y_scaler.transform(y).ravel()  # 1D target

        train_x = torch.from_numpy(Xn).to(device)
        train_y = torch.from_numpy(yn).to(device)

        # Build GP
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
        self.model = ExactGPModel(
            train_x, train_y, self.likelihood,
            kernel=self.kernel, ard_dims=X.shape[-1]
        ).to(device)

        self.model.train()
        self.likelihood.train()

        # Optimizer + marginal likelihood
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)

        print(f"[GPManager] Training GP ({self.kernel}) on {X.shape[0]} points.")
        for i in range(self.iters):
            optimizer.zero_grad()
            output = self.model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()

            if (i + 1) % 50 == 0 or i == 0 or i == self.iters - 1:
                print(f"  iter {i+1:4d}/{self.iters} | loss = {loss.item():.4f}")

        print("[GPManager] Training done.")

    def append_and_fit(self, X_new: np.ndarray, y_new: np.ndarray, max_points: int | None = None):
        """
        Append new (X_new, y_new) to existing dataset and retrain.
        If max_points is not None, keep only the most recent max_points in the buffer.
        """
        X_new = np.asarray(X_new, dtype=np.float32)
        y_new = np.asarray(y_new, dtype=np.float32).reshape(-1, 1)

        if self.X_all is None:
            X_cat = X_new
            Y_cat = y_new
        else:
            X_cat = np.concatenate([self.X_all, X_new], axis=0)
            Y_cat = np.concatenate([self.Y_all, y_new], axis=0)

        if max_points is not None and X_cat.shape[0] > max_points:
            X_cat = X_cat[-max_points:]
            Y_cat = Y_cat[-max_points:]

        print(f"[GPManager] Retrain buffer size={X_cat.shape[0]} (new={X_new.shape[0]}).")
        self.fit(X_cat, Y_cat)

    @torch.no_grad()
    def predict(self, X_query: np.ndarray):
        """
        Predict mean and variance for X_query (numpy).
        Returns: mean (N,), var (N,)
        """
        assert self.model is not None, "GP not trained yet."

        self.model.eval()
        self.likelihood.eval()

        Xq = np.asarray(X_query, dtype=np.float32)
        Xq_n = self.x_scaler.transform(Xq)
        x_t = torch.from_numpy(Xq_n).to(device)

        with gpytorch.settings.fast_pred_var():
            pred = self.likelihood(self.model(x_t))

        mean_n = pred.mean.detach().cpu().numpy()
        var_n = pred.variance.detach().cpu().numpy()

        # Un-normalize mean; adjust variance for scaling
        mean = self.y_scaler.inverse_transform(mean_n.reshape(-1, 1)).ravel()
        # If y = s * y_n + m, then Var[y] = s^2 * Var[y_n]
        y_std = self.y_scaler.std.ravel()[0]
        var = (y_std ** 2) * var_n
        return mean.astype(np.float32), var.astype(np.float32)


# ---------- Train two GPs: for Δphi and Δomega ----------
gps = [
    GPManager(kernel='RBF', lr=0.05, iters=600, seed=0),  # Δphi
    GPManager(kernel='RBF', lr=0.05, iters=600, seed=1),  # Δomega
]

# Δphi GP
gps[0].fit(X_np, Y_np[:, 0])
# Δomega GP
gps[1].fit(X_np, Y_np[:, 1])

print("[Cell 5] Trained two GP models for Δphi and Δomega.")


# ---------- Numpy wrapper: single-step dynamics ----------
def gp_predict_delta_numpy(X_query: np.ndarray):
    """
    Given X_query (N, 2) = [phi, omega_clipped],
    return:
        dphi_mean (N,), dphi_var (N,),
        domega_mean (N,), domega_var (N,)
    All numpy float32 arrays.
    """
    dphi_mean, dphi_var = gps[0].predict(X_query)
    domega_mean, domega_var = gps[1].predict(X_query)
    return dphi_mean, dphi_var, domega_mean, domega_var


# ---------- Meta info we may want later (for MPPI, etc.) ----------
# dt_ctrl and OMEGA_CLIP_RAD should exist in globals from previous cells,
# but we fall back to values from the saved npz if needed.
dt_ctrl_meta = None
omega_clip_meta = None

if 'dt_ctrl' in globals():
    dt_ctrl_meta = float(dt_ctrl)
else:
    try:
        data_meta = np.load("monstertruck_gp_initial_data.npz")
        dt_ctrl_meta = float(data_meta.get("dt_ctrl", 0.01))
    except Exception:
        dt_ctrl_meta = 0.01

if 'OMEGA_CLIP_RAD' in globals():
    omega_clip_meta = float(OMEGA_CLIP_RAD)
else:
    try:
        data_meta = np.load("monstertruck_gp_initial_data.npz")
        omega_clip_meta = float(np.deg2rad(data_meta.get("omega_clip_deg", 200.0)))
    except Exception:
        omega_clip_meta = float(math.radians(200.0))

gp_data_meta = dict(
    dt_ctrl=float(dt_ctrl_meta),
    omega_clip_rad=float(omega_clip_meta),
    N_train=int(N),
)

print("[Cell 5] gp_predict_delta_numpy(...) and gp_data_meta ready.")


In [ ]:
# ============================================================
# Cell 6 — GP slice (Δφ) + data distribution over φ
#   - Plot 1: GP mean ± 2σ vs φ at fixed ω
#   - Plot 2: Histogram of collected φ over [-180°, 180°]
# ============================================================
import numpy as np
import matplotlib.pyplot as plt

# --------------------------------------------------------
# Choose GP: Δφ model
# --------------------------------------------------------
gp = gps[0]        # GP for Δφ

# Fix angular rate for the slice
omega_fixed = 0.0      # rad/s (≈ 0 deg/s)

# --------------------------------------------------------
# Define φ grid (in radians)
# --------------------------------------------------------
phi_min, phi_max = -np.pi, np.pi   # full range [-180°, +180°]
phi_grid = np.linspace(phi_min, phi_max, 400, dtype=np.float32)

# Construct query points [φ, ω_fixed]
X_query = np.column_stack([
    phi_grid,
    np.full_like(phi_grid, omega_fixed, dtype=np.float32),
])

# --------------------------------------------------------
# Predict GP mean and std (in radians)
# --------------------------------------------------------
Mean_rad, Var = gp.predict(X_query)
Var = np.maximum(Var, 0.0)        # numerical safety
Std_rad = np.sqrt(Var)

# Convert to degrees for plotting
phi_grid_deg = np.rad2deg(phi_grid)
Mean_deg     = np.rad2deg(Mean_rad)
Std_deg      = np.rad2deg(Std_rad)

# --------------------------------------------------------
# Overlay training data near the chosen ω slice
# --------------------------------------------------------
X_all = gp.X_all         # shape (N, 2): [φ, ω_clipped]
Y_all = gp.Y_all         # shape (N, 1): Δφ in radians

if X_all is None or Y_all is None:
    print("[Cell 6] No training data stored in gp.X_all / gp.Y_all.")
    phi_train_deg  = np.array([])
    dphi_train_deg = np.array([])
else:
    Y_all_rad = Y_all.ravel()

    # Select points close to the chosen slice in ω
    omega_tol_deg = 5.0          # +/- 5 deg/s around omega_fixed

    mask = (
        np.abs(np.rad2deg(X_all[:, 1] - omega_fixed)) < omega_tol_deg
    )

    phi_train_deg  = np.rad2deg(X_all[mask, 0])
    dphi_train_deg = np.rad2deg(Y_all_rad[mask])

# --------------------------------------------------------
# Plot 1: GP mean ± 2σ as shaded region
# --------------------------------------------------------
plt.figure(figsize=(8, 5))
plt.plot(phi_grid_deg, Mean_deg, lw=2, label='GP mean Δφ (deg)')

plt.fill_between(
    phi_grid_deg,
    Mean_deg - 2 * Std_deg,
    Mean_deg + 2 * Std_deg,
    alpha=0.2,
    label='±2σ (uncertainty)'
)

# Overlay training data
if phi_train_deg.size > 0:
    plt.scatter(
        phi_train_deg,
        dphi_train_deg,
        s=25,
        alpha=0.7,
        label=f'training data (|ω-ω_fix|<{omega_tol_deg}°)'
    )
else:
    print("[Cell 6] No training points matched the ω slice mask.")

plt.xlabel("flip angle φ (deg)  —  0 = upside-down, ±180 = upright")
plt.ylabel("Δφ per control step (deg)")
plt.title(f"GP slice: Δφ vs φ  (ω={np.rad2deg(omega_fixed):.1f} deg/s)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# ============================================================
# Plot 2 — Data distribution over flip angle φ (deg)
# ============================================================

# Get φ for histogram
if X_all is not None:
    phi_all = X_all[:, 0].astype(np.float32)   # radians
    print("[Cell 6] Using gp.X_all[:, 0] for φ distribution.")
elif 'phi_arr' in globals():
    phi_all = phi_arr.astype(np.float32)
    print("[Cell 6] Using phi_arr from collection for φ distribution.")
elif 'X_np' in globals():
    phi_all = X_np[:, 0].astype(np.float32)
    print("[Cell 6] Using X_np[:, 0] for φ distribution.")
else:
    data_npz = np.load("monstertruck_gp_initial_data.npz")
    phi_all = data_npz["phi"].astype(np.float32)
    print("[Cell 6] Loaded phi from 'monstertruck_gp_initial_data.npz' for distribution.")

phi_deg_all = np.rad2deg(phi_all)
phi_min_all, phi_max_all = phi_deg_all.min(), phi_deg_all.max()
N_hist = phi_deg_all.shape[0]

print(f"[Cell 6] N samples (for histogram) = {N_hist}")
print(f"[Cell 6] φ range (deg) ≈ [{phi_min_all:+.2f}, {phi_max_all:+.2f}]")

# Histogram over [-180, 180]
bins = 72  # 5° bins: 360 / 5 = 72
bin_edges = np.linspace(-180.0, 180.0, bins + 1)

plt.figure(figsize=(8, 4))
plt.hist(phi_deg_all, bins=bin_edges, alpha=0.85, edgecolor='k', linewidth=0.4)

plt.xlabel("flip angle φ (deg)  —  0 = upside-down, ±180 = upright")
plt.ylabel("count")
plt.title("Distribution of collected data over flip angle φ")
plt.xlim([-180, 180])
plt.grid(True, alpha=0.3, linestyle="--", linewidth=0.7)
plt.tight_layout()
plt.show()


|GP Training Script

In [ ]:
# ============================================================
# Cell 7 — GP-based dynamics + task & exploration costs
#   - GP models: X = [phi, omega_clipped] -> [Δphi, Δomega]
#   - No action input to GP (dynamics depend only on state)
# ============================================================
import numpy as np

# ---------- Helpers ----------
def clip_angle_pi(phi):
    """Wrap any angle to (-pi, pi]."""
    return ((phi + np.pi) % (2.0 * np.pi)) - np.pi


def is_upright_success(phi, omega):
    """
    Success condition in radians, using global thresholds from Cell 1:
      - abs(phi) >= SUCCESS_ENTER_RAD
      - abs(omega) <= MAX_UPRIGHT_RATE_RAD
    """
    return (abs(phi) >= SUCCESS_ENTER_RAD) and (abs(omega) <= MAX_UPRIGHT_RATE_RAD)


# ---------- GP-based single-step dynamics ----------
def gp_step(state, action):
    """
    One-step prediction using the learned GP dynamics.

    Inputs:
      state  : [phi, omega] at time t  (radians, rad/s)
      action : throttle u_t in [-1, 1] (NOT used by the GP; kept for cost terms)

    The GP was trained on:
      X = [phi, omega_clipped]
      Y = [Δphi, Δomega]

    Returns:
      next_state : [phi_next, omega_next]
      var_total  : scalar uncertainty measure = Var(Δphi) + Var(Δomega)
    """
    phi, omega = float(state[0]), float(state[1])

    # Clip omega exactly the same way as in data collection (Cell 3)
    omega_in = float(np.clip(
        omega,
        -gp_data_meta["omega_clip_rad"],
        +gp_data_meta["omega_clip_rad"]
    ))

    # Query GP with [phi, omega_clipped]
    X_query = np.array([[phi, omega_in]], dtype=np.float32)  # shape (1, 2)

    dphi_mean, dphi_var, domega_mean, domega_var = gp_predict_delta_numpy(X_query)

    dphi_m   = float(dphi_mean[0])
    domega_m = float(domega_mean[0])
    v_phi    = float(dphi_var[0])
    v_omega  = float(domega_var[0])

    # Discrete-time update over one control interval dt_ctrl
    phi_next   = clip_angle_pi(phi + dphi_m)
    omega_next = omega + domega_m

    # Aggregate uncertainty for exploration bonus
    var_total = v_phi + v_omega

    next_state = np.array([phi_next, omega_next], dtype=np.float32)
    return next_state, var_total


# ---------- Exploration cost ----------
def explore_cost_step(var_total, eps=1e-8):
    """
    Exploration cost: -log(variance).
    Higher predictive variance -> more negative cost (i.e., better for exploration
    when added with a positive weight).
    """
    v = max(float(var_total), eps)
    return -np.log(v)


# ---------- Task cost weights ----------
TASK_COST_WEIGHTS = dict(
    w_angle=200.0,                    # weight on angle error
    w_rate=20.0,                      # weight on angular rate
    w_jerk=1.0,                       # weight on control change Δu
    w_time=1.5,                       # per-step time penalty
    angle_norm=np.pi,                 # normalize angle error by π
    omega_ref=np.deg2rad(120.0),      # reference rate for scaling
)


def task_cost_step(state, action, w=TASK_COST_WEIGHTS):
    """
    Task cost encouraging:
      - being upright (φ ≈ ±π, away from upside-down φ ≈ 0)
      - reasonable angular rates, especially near upright
      - small control jerk (handled separately in rollout)
    """
    phi, omega = float(state[0]), float(state[1])

    # Angle term: distance from upright (π away from upside-down)
    angle_err = np.pi - min(abs(phi), np.pi)       # 0 when φ at ±π, max at 0
    angle_dist = (angle_err / w["angle_norm"])     # normalized to [0, 1]
    angle_dist = np.clip(angle_dist, 0.0, 1.0)

    c_angle = w["w_angle"] * (angle_dist**2)

    # Rate term: only penalize large |ω| when closer to upright
    closeness = 1.0 - angle_dist                   # 0 near UD, 1 near upright
    omega_norm = omega / w["omega_ref"]
    c_rate = w["w_rate"] * closeness * (omega_norm**2)

    # Time penalty
    c_time = w["w_time"]

    return c_angle + c_rate + c_time


def rollout_cost_from_actions(
    state0,
    actions,
    wt_task=1.0,
    wt_explore=0.0,
    gamma=0.99,
    early_stop_on_success=True
):
    """
    Rollout cost for a candidate action sequence using the GP dynamics.

    Inputs:
      state0    : initial state [phi, omega]
      actions   : sequence of scalar throttles (length H)
      wt_task   : weight on task cost sum
      wt_explore: weight on exploration bonus sum
      gamma     : discount factor
      early_stop_on_success: if True, stop when upright success is reached

    Returns:
      total_cost, J_task, J_expl
    """
    state = np.array(state0, dtype=np.float32)
    H = len(actions)

    J_task = 0.0
    J_expl = 0.0
    disc = 1.0

    u_prev = 0.0

    for t in range(H):
        u_t = float(actions[t])

        # GP prediction for next state (dynamics independent of u_t)
        next_state, var_total = gp_step(state, u_t)

        # Task cost at current state
        c_task_core = task_cost_step(state, u_t)

        # Jerk penalty on control changes
        du = u_t - u_prev
        c_jerk = TASK_COST_WEIGHTS["w_jerk"] * (du**2)

        c_task = c_task_core + c_jerk

        # Exploration cost from model uncertainty
        c_expl = explore_cost_step(var_total)

        J_task += disc * c_task
        J_expl += disc * c_expl

        state = next_state
        disc *= gamma
        u_prev = u_t

        # Optional early stopping when upright
        if early_stop_on_success and is_upright_success(state[0], state[1]):
            break

    total_cost = wt_task * J_task + wt_explore * J_expl
    return float(total_cost), float(J_task), float(J_expl)


In [ ]:
# ============================================================
# Cell 8 — MPPI controller + online GP retrain loop (+ viz)
#   - MPPI uses ONLY GP dynamics (gp_step / rollout_cost_from_actions)
#   - Outer loop:
#       * From current real state (MuJoCo), run MPPI in GP model
#       * Execute ONLY the best action u* in MuJoCo
#       * Log real transition (phi, omega -> dphi, domega)
#       * Pass through SAME occupancy bins as Cell 3
#       * If bin not full, append to GP dataset
#       * Retrain GPs every RETRAIN_EVERY_NEW appended samples
#         and decay exploration weight each retrain
#       * On each retrain:
#           - print N used in GP
#           - show GP Δφ slice plot + (φ, ω) bin occupancy plot
#       * Stop when real truck is upright (is_upright_success)
# ============================================================
import os
import time
import numpy as np
import mujoco
import matplotlib.pyplot as plt

# ---------- MPPI controller ----------
class MPPIController:
    def __init__(
        self,
        H=100,
        K=256,
        sigma=0.6,
        lam=1.0,
        gamma=0.99,
        wt_task=1.0,
        wt_explore=0.0,
        action_low=-1.0,
        action_high=1.0,
        seed=0,
    ):
        self.H = H
        self.K = K
        self.sigma = float(sigma)
        self.lam = float(lam)
        self.gamma = float(gamma)
        self.wt_task = float(wt_task)
        self.wt_explore = float(wt_explore)
        self.action_low = float(action_low)
        self.action_high = float(action_high)

        self.rng = np.random.default_rng(seed)
        # MPPI base sequence ū (shifted every step)
        self.base_actions = np.zeros(self.H, dtype=np.float32)

    def _sample_action_sequences(self):
        noise = self.rng.normal(
            loc=0.0,
            scale=self.sigma,
            size=(self.K, self.H)
        ).astype(np.float32)

        U = self.base_actions[None, :] + noise  # (K, H)
        U = np.clip(U, self.action_low, self.action_high)
        return U

    def act(self, state, u_prev=0.0):
        """
        Run MPPI from the given starting state using the GP dynamics.
        Returns:
          u0  : first control in the weighted sequence
          info: diagnostic dict
        """
        state0 = np.array(state, dtype=np.float32).copy()
        u_prev = float(u_prev)

        U = self._sample_action_sequences()  # (K, H)

        costs = np.zeros(self.K, dtype=np.float64)
        J_task_arr = np.zeros(self.K, dtype=np.float64)
        J_expl_arr = np.zeros(self.K, dtype=np.float64)

        for k in range(self.K):
            a_seq = U[k]

            cost, J_task, J_expl = rollout_cost_from_actions(
                state0,
                a_seq,
                wt_task=self.wt_task,
                wt_explore=self.wt_explore,
                gamma=self.gamma,
                early_stop_on_success=True,
            )
            costs[k] = cost
            J_task_arr[k] = J_task
            J_expl_arr[k] = J_expl

        # MPPI weighting
        c_min = costs.min()
        weights = np.exp(-(costs - c_min) / self.lam)
        w_sum = np.sum(weights) + 1e-12
        weights /= w_sum

        # Control to apply now
        u0 = float(np.sum(weights * U[:, 0]))
        u0 = float(np.clip(u0, self.action_low, self.action_high))

        # Update base sequence (shifted mean)
        mean_seq = np.sum(weights[:, None] * U, axis=0)  # (H,)
        new_base = np.zeros_like(self.base_actions)
        new_base[:-1] = mean_seq[1:]
        new_base[-1] = 0.0
        self.base_actions = new_base.astype(np.float32)

        info = dict(
            costs=costs,
            J_task=J_task_arr,
            J_explore=J_expl_arr,
            cost_min=float(c_min),
            cost_mean=float(costs.mean()),
            cost_std=float(costs.std()),
            u0=u0,
        )
        return u0, info


def JointMPPI(explore_weight,
              H=20,
              K=100,
              sigma=0.6,
              lam=1.0,
              gamma=0.99,
              seed=0):
    """
    Convenience constructor: single MPPI where
      cost = J_task + explore_weight * J_expl
    """
    return MPPIController(
        H=H,
        K=K,
        sigma=sigma,
        lam=lam,
        gamma=gamma,
        wt_task=1.0,
        wt_explore=float(explore_weight),
        action_low=ACTION_LOW,
        action_high=ACTION_HIGH,
        seed=seed,
    )


# ---------- Helper: per-retrain visualization ----------
def visualize_gp_slice_and_bins(gp, retrain_idx,
                                omega_fixed=0.0,
                                omega_tol_deg=5.0,
                                n_bins_phi=40,
                                n_bins_omega=40):
    """
    gp:          GPManager instance for Δφ (gps[0])
    retrain_idx: integer index of retrain (1, 2, 3, ...)
    """

    if gp.X_all is None or gp.Y_all is None:
        print(f"[Viz] GP has no stored data yet; skipping plots for retrain #{retrain_idx}.")
        return

    X_all = gp.X_all          # (N, 2): [phi, omega_clipped]
    Y_all = gp.Y_all.ravel()  # (N,): Δφ (rad)

    N = X_all.shape[0]

    # ------------------ 1) GP slice: Δφ vs φ at fixed ω ------------------
    phi_min, phi_max = -np.pi, np.pi
    phi_grid = np.linspace(phi_min, phi_max, 400, dtype=np.float32)

    # Query along slice (φ varies, ω = omega_fixed)
    X_query = np.column_stack([
        phi_grid,
        np.full_like(phi_grid, omega_fixed, dtype=np.float32),
    ])

    Mean_rad, Var = gp.predict(X_query)
    Var = np.maximum(Var, 0.0)
    Std_rad = np.sqrt(Var)

    phi_grid_deg = np.rad2deg(phi_grid)
    Mean_deg     = np.rad2deg(Mean_rad)
    Std_deg      = np.rad2deg(Std_rad)

    # Training points near the slice in ω
    mask = np.abs(X_all[:, 1] - omega_fixed) < np.deg2rad(omega_tol_deg)
    phi_train_deg  = np.rad2deg(X_all[mask, 0])
    dphi_train_deg = np.rad2deg(Y_all[mask])

    # ------------------ 2) Bin / occupancy over (φ, ω) ------------------
    phi_all_deg   = np.rad2deg(X_all[:, 0])
    omega_all_deg = np.rad2deg(X_all[:, 1])

    PHI_MIN_DEG, PHI_MAX_DEG = -180.0, 180.0
    OMEGA_MIN_DEG, OMEGA_MAX_DEG = -200.0, 200.0

    H2d, xedges, yedges = np.histogram2d(
        phi_all_deg,
        omega_all_deg,
        bins=[n_bins_phi, n_bins_omega],
        range=[[PHI_MIN_DEG, PHI_MAX_DEG], [OMEGA_MIN_DEG, OMEGA_MAX_DEG]],
    )

    # ------------------ Make figure ------------------
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(
        f"Retrain #{retrain_idx}: GP Δφ slice + bin occupancy  (N = {N} samples)",
        fontsize=12
    )

    # ---- Left: GP mean ± 2σ slice ----
    ax0 = axes[0]
    ax0.plot(phi_grid_deg, Mean_deg, lw=2, label='GP mean Δφ (deg)')
    ax0.fill_between(
        phi_grid_deg,
        Mean_deg - 2 * Std_deg,
        Mean_deg + 2 * Std_deg,
        alpha=0.2,
        label='±2σ (uncertainty)',
    )

    if phi_train_deg.size > 0:
        ax0.scatter(
            phi_train_deg,
            dphi_train_deg,
            s=25,
            alpha=0.7,
            label=f'training data (|ω-ω_fix|<{omega_tol_deg}°)',
        )
    else:
        print(f"[Viz] Retrain #{retrain_idx}: no training points near ω={np.rad2deg(omega_fixed):.1f}°/s slice.")

    ax0.set_xlabel("flip angle φ (deg) — 0 = upside-down, ±180 = upright")
    ax0.set_ylabel("Δφ per control step (deg)")
    ax0.set_title(f"Δφ vs φ slice at ω = {np.rad2deg(omega_fixed):.1f} deg/s")
    ax0.grid(True, alpha=0.3)
    ax0.legend()

    # ---- Right: 2D bin / occupancy ----
    ax1 = axes[1]
    im = ax1.imshow(
        H2d.T,
        origin="lower",
        aspect="auto",
        extent=[PHI_MIN_DEG, PHI_MAX_DEG, OMEGA_MIN_DEG, OMEGA_MAX_DEG],
    )
    ax1.set_xlabel("φ (deg)")
    ax1.set_ylabel("ω (deg/s)")
    ax1.set_title("Bin occupancy in (φ, ω) space")
    fig.colorbar(im, ax=ax1, label="count per bin")

    fig.tight_layout()
    plt.show()


# ---------- Online MPPI + GP retrain loop ----------
def run_mppi_gp_retrain_loop(
    max_real_steps=500,
    retrain_every_new=10,
    explore_weight_init=5.0,   # start exploration weight
    explore_decay=0.7,         # multiply wt_explore after each retrain
    H=20,
    K=100,
    sigma=0.6,
    lam=1.0,
    gamma=0.99,
    seed=0,
    render=False,
):
    """
    Online learning loop (with sparse logging every 100 real steps).
    Exploration weight starts at explore_weight_init and decays
    by factor explore_decay after each GP retrain.
    """

    LOG_EVERY_REAL = 100   # print progress every N real steps

    # ---- Make sure XML exists ----
    if not os.path.exists(XML_PATH):
        raise FileNotFoundError(f"[Cell 8] Cannot find XML_PATH: {XML_PATH}")

    # ---- Build MuJoCo model + data ----
    model = mujoco.MjModel.from_xml_path(XML_PATH)
    data = mujoco.MjData(model)

    # Control timing & flip tracker
    dt_ctrl = model.opt.timestep * FRAME_SKIP
    print(f"[Cell 8] dt_ctrl (real) = {dt_ctrl:.4f} s")

    flip_tracker = make_flip_tracker(model, frame_skip=FRAME_SKIP)

    # Body / actuators
    chassis_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "chassis")
    if chassis_id == -1:
        chassis_id = 0
    throttle_ids = find_throttle_ids(model)

    print(f"[Cell 8] Driving actuators in online loop: "
          f"{[mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR, i) for i in throttle_ids]}")

    # ---- Optional render (simple GLFW viewer) ----
    if render:
        from mujoco.glfw import glfw
        if not glfw.init():
            raise RuntimeError("GLFW init failed")
        window = glfw.create_window(1000, 800, "MonsterTruck — MPPI online training", None, None)
        if not window:
            glfw.terminate()
            raise RuntimeError("GLFW window creation failed")
        glfw.make_context_current(window)
        glfw.swap_interval(1)
        cam = mujoco.MjvCamera()
        opt = mujoco.MjvOption()
        scene = mujoco.MjvScene(model, maxgeom=10000)
        context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)
        cam.distance, cam.elevation, cam.azimuth = 3.0, -25, 90
    else:
        window = None

    # ---- Reset env and get initial (phi, omega) ----
    reset_state(model, data)
    flip_tracker.reset()
    mujoco.mj_forward(model, data)

    phi_prev, omega_prev, _ = flip_tracker.step(data, chassis_id, model=model)
    state = np.array([phi_prev, omega_prev], dtype=np.float32)

    print(f"[Cell 8] Initial real state: "
          f"phi={np.rad2deg(phi_prev):+.2f} deg, "
          f"omega={np.rad2deg(omega_prev):+.2f} deg/s")

    # ---- Create MPPI controller ----
    mppi = MPPIController(
        H=H,
        K=K,
        sigma=sigma,
        lam=lam,
        gamma=gamma,
        wt_task=1.0,
        wt_explore=float(explore_weight_init),
        action_low=ACTION_LOW,
        action_high=ACTION_HIGH,
        seed=seed,
    )

    # ---- Buffers for NEW samples only ----
    new_X = []       # rows [phi_prev, omega_clipped_prev]
    new_dphi = []    # Δφ
    new_domega = []  # Δω

    appended_since_retrain = 0
    total_accepted_new = 0
    retrain_idx = 0

    start_wall = time.perf_counter()

    for step_idx in range(max_real_steps):
        # 1) MPPI in GP model to choose best action from current state
        u_star, info = mppi.act(state)

        # 2) Execute u_star ONCE in real MuJoCo
        apply_throttle(data, throttle_ids, u_star)
        for _ in range(FRAME_SKIP):
            mujoco.mj_step(model, data)

        # 3) Read new real state via flip tracker
        phi_curr, omega_curr, _ = flip_tracker.step(data, chassis_id, model=model)

        # Compute deltas
        dphi = angdiff_rad(phi_curr, phi_prev)
        domega = omega_curr - omega_prev

        # Clipped omega for GP input (same as training)
        omega_clip_rad = gp_data_meta["omega_clip_rad"]
        omega_clipped_prev = float(np.clip(omega_prev, -omega_clip_rad, +omega_clip_rad))

        # 4) Bin (phi_prev, omega_clipped_prev) and decide whether to save
        i_phi, i_om = state_to_bin(phi_prev, omega_clipped_prev)
        bin_count = occupancy[i_phi, i_om]

        accepted = False
        if bin_count < MAX_SAMPLES_PER_BIN:
            occupancy[i_phi, i_om] += 1
            new_X.append([phi_prev, omega_clipped_prev])
            new_dphi.append(dphi)
            new_domega.append(domega)
            appended_since_retrain += 1
            total_accepted_new += 1
            accepted = True

        # 5) Retrain every 'retrain_every_new' accepted samples
        if appended_since_retrain >= retrain_every_new:
            X_new = np.asarray(new_X, dtype=np.float32)
            Y_new = np.column_stack([
                np.asarray(new_dphi, dtype=np.float32),
                np.asarray(new_domega, dtype=np.float32),
            ])

            retrain_idx += 1
            print(f"\n[Cell 8] Retrain #{retrain_idx} with {X_new.shape[0]} new accepted samples "
                  f"(total accepted so far={total_accepted_new}).")

            # Δφ GP
            gps[0].append_and_fit(X_new, Y_new[:, 0])
            # Δω GP
            gps[1].append_and_fit(X_new, Y_new[:, 1])

            # How many data are we using now?
            N_used = gps[0].X_all.shape[0] if gps[0].X_all is not None else 0
            print(f"[Cell 8] Retrain #{retrain_idx}: GP dataset size now N = {N_used}")

            # ---- Decay exploration weight after retrain ----
            old_explore = mppi.wt_explore
            mppi.wt_explore *= float(explore_decay)
            print(f"[Cell 8] Decayed wt_explore: {old_explore:.3f} -> {mppi.wt_explore:.3f}")

            # Visualization: GP Δφ slice + bin occupancy
            visualize_gp_slice_and_bins(gps[0], retrain_idx)

            new_X.clear()
            new_dphi.clear()
            new_domega.clear()
            appended_since_retrain = 0

        # Optional: render
        if render and window is not None:
            from mujoco.glfw import glfw
            cam.lookat[:] = data.xpos[chassis_id]
            w, h = glfw.get_framebuffer_size(window)
            if w > 0 and h > 0:
                mujoco.mjv_updateScene(model, data, opt, None, cam,
                                       mujoco.mjtCatBit.mjCAT_ALL, scene)
                viewport = mujoco.MjrRect(0, 0, w, h)
                mujoco.mjr_render(viewport, scene, context)
            glfw.swap_buffers(window)
            glfw.poll_events()
            if glfw.window_should_close(window):
                print("[Cell 8] Viewer closed; stopping loop.")
                break

        # Progress log (ONLY every LOG_EVERY_REAL steps)
        if (step_idx % LOG_EVERY_REAL) == 0:
            elapsed = time.perf_counter() - start_wall
            print(
                f"[Cell 8] step={step_idx:4d}  u*={u_star:+.3f}  "
                f"phi={np.rad2deg(phi_curr):+7.2f} deg  "
                f"omega={np.rad2deg(omega_curr):+7.2f} deg/s  "
                f"accepted={accepted}  "
                f"total_new={total_accepted_new}  "
                f"elapsed={elapsed:.1f}s"
            )

        # 6) Check SUCCESS on REAL state
        if is_upright_success(phi_curr, omega_curr):
            print(
                f"[Cell 8] SUCCESS at real step {step_idx}: "
                f"phi={np.rad2deg(phi_curr):.2f} deg, "
                f"omega={np.rad2deg(omega_curr):.2f} deg/s, "
                f"total_new={total_accepted_new}"
            )
            break

        # Prepare for next control step
        phi_prev, omega_prev = phi_curr, omega_curr
        state = np.array([phi_prev, omega_prev], dtype=np.float32)

    else:
        print(
            f"[Cell 8] Reached max_real_steps={max_real_steps} WITHOUT success. "
            f"Total accepted new samples={total_accepted_new}."
        )

    if render and window is not None:
        from mujoco.glfw import glfw
        glfw.terminate()
        print("[Cell 8] Viewer terminated.")

    return model, data, mppi


print("[Cell 8] MPPIController, JointMPPI(...) and run_mppi_gp_retrain_loop(...) are ready.")


In [ ]:
# ============================================================
# Cell 9 — Run MPPI + online GP retrain loop (with rendering)
# ============================================================

# ---- Hyperparameters for the online loop ----
MAX_REAL_STEPS        = 400      # max real control steps before giving up
RETRAIN_EVERY_NEW     = 10       # retrain every N accepted new samples

# Exploration schedule: start high, decay each retrain
EXPLORE_WEIGHT_INIT   = 5.0      # initial exploration weight
EXPLORE_DECAY         = 0.7      # multiply by this after each retrain

MPPI_H                = 20       # planning horizon for MPPI
MPPI_K                = 120      # number of sampled trajectories
MPPI_SIGMA            = 0.6      # action noise std
MPPI_LAMBDA           = 1.0      # temperature
MPPI_GAMMA            = 0.99     # discount
MPPI_SEED             = 0

# *** TURN RENDERING ON HERE ***
RENDER_ONLINE         = True     # <-- set to True to see the truck

print("[Cell 9] Starting online MPPI + GP retrain loop...")
print(f"[Cell 9]   MAX_REAL_STEPS      = {MAX_REAL_STEPS}")
print(f"[Cell 9]   RETRAIN_EVERY_NEW   = {RETRAIN_EVERY_NEW}")
print(f"[Cell 9]   EXPLORE_WEIGHT_INIT = {EXPLORE_WEIGHT_INIT}")
print(f"[Cell 9]   EXPLORE_DECAY       = {EXPLORE_DECAY}")
print(f"[Cell 9]   MPPI: H={MPPI_H}, K={MPPI_K}, sigma={MPPI_SIGMA}, "
      f"lambda={MPPI_LAMBDA}, gamma={MPPI_GAMMA}")

model_online, data_online, mppi_online = run_mppi_gp_retrain_loop(
    max_real_steps=MAX_REAL_STEPS,
    retrain_every_new=RETRAIN_EVERY_NEW,
    explore_weight_init=EXPLORE_WEIGHT_INIT,
    explore_decay=EXPLORE_DECAY,
    H=MPPI_H,
    K=MPPI_K,
    sigma=MPPI_SIGMA,
    lam=MPPI_LAMBDA,
    gamma=MPPI_GAMMA,
    seed=MPPI_SEED,
    render=RENDER_ONLINE,   # <<=== this makes the MuJoCo viewer appear
)

print("[Cell 9] Online MPPI + GP retrain loop finished.")
